In [ ]:
import numpy as np
import argparse
import os
import tensorflow as tf
from PIL import Image
from io import BytesIO
import glob
import csv
import matplotlib.pyplot as plt

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
import tensorflow as tf
import numpy as np

def load_tfrecord_dataset(tfrecord_path):
    # Define the feature description dictionary for parsing the TFRecord file
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
    }

    def _parse_tfrecord(example_proto):
        # Parse a single example from the TFRecord file
        return tf.io.parse_single_example(example_proto, feature_description)

    # Create a dataset from the TFRecord file
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(_parse_tfrecord)

    return dataset

def evaluate_model(dataset, model, iou_threshold):
    true_positives = 0
    false_positives = 0

    for data in dataset:
        image = data['image/encoded']
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.expand_dims(image, 0)
        # Preprocess the image if required (e.g., resize, normalization)

        # Make predictions with your object detection model
        predictions = model.predict(image)

        # Extract the predicted bounding boxes from the model's output if required

        # Compare the predicted bounding boxes with the ground truth bounding boxes
        iou_max = 0
        for pred_box in predictions:
            for i in range(data['image/object/bbox/xmin'].values.shape[0]):
                gt_box = [
                    data['image/object/bbox/xmin'].values[i],
                    data['image/object/bbox/ymin'].values[i],
                    data['image/object/bbox/xmax'].values[i],
                    data['image/object/bbox/ymax'].values[i]
                ]
                iou = calculate_iou(gt_box, pred_box)
                if iou > iou_max:
                    iou_max = iou

        # If the best IoU is above the threshold, count it as a true positive
        if iou_max >= iou_threshold:
            true_positives += 1
        else:
            false_positives += 1

    # Calculate precision
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

    return precision

# Example usage
tfrecord_path = 'path/to/your/test_dataset.tfrecord'
iou_threshold = 0.5
model_path = 'path/to/your/model.pb'

# Load the TFRecord dataset
dataset = load_tfrecord_dataset(tfrecord_path)

# Load the model
model = tf.keras.models.load_model(model_path)

# Evaluate the model on the dataset and calculate precision
precision = evaluate_model(dataset, model, iou_threshold)
print("Precision:", precision)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

# set paths and filenames
model_path = 'E:\Major_project_main\Main\inference_graph3\saved_model\saved_model.pb'
test_data_dir = 'E:\Major_project_main\Data\final_data\test'
test_csv_path = 'E:\Major_project_main\Data\final_data\test.csv'

# set model input and output names
input_name = 'input_1'
output_names = ['efficientdet-d0/filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0']

# load the model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# load the test data CSV
test_df = pd.read_csv(test_csv_path)

# create a list to hold the results
results = []

# loop through the test data and evaluate the model
for index, row in test_df.iterrows():
    # load the image
    image_path = os.path.join(test_data_dir, row['name'])
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(512, 512))
    input_data = np.expand_dims(tf.keras.preprocessing.image.img_to_array(image), axis=0)

    # set the model input and invoke the interpreter
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)
    interpreter.invoke()

    # get the model output and post-process   
    output_data = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
    detections = output_data[0]

    # filter out low confidence detections
    confidence_threshold = 0.5
    detections = detections[detections[:, 4] >= confidence_threshold]

    # convert from normalized coordinates to pixel coordinates
    height, width, _ = image.shape
    detections[:, 0] *= width
    detections[:, 1] *= height
    detections[:, 2] *= width
    detections[:, 3] *= height

    # convert to integers and add to the results list
    for detection in detections:
        results.append({
            'image_id': row['name'],
            'width': row['width'],
            'height': row['height'],
            'label': 'cricket ball',
            'confidence': detection[4],
            'xmin': int(detection[0]),
            'ymin': int(detection[1]),
            'xmax': int(detection[2]),
            'ymax': int(detection[3]),
        })

# create a new DataFrame with the results and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('E:\Major_project_main\Data\final_data\test\results.csv', index=False)

ValueError: 